In [1]:
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(pow(2,40))
import cv2
import imageio
# import matplotlib.pyplot as plt
import numpy as np
import pathlib
import glob
import pandas as pd
from tqdm import tqdm
# from pyometiff import OMETIFFReader
from tiatoolbox.tools import patchextraction
from tiatoolbox.tools.tissuemask import OtsuTissueMasker
import imageio.v2 as imageio

|2024-04-17|10:34:08.563| [WARNING] /mnt/cephfs/home/users/fshahi/miniconda3/envs/mif/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



In [14]:
ll '/mnt/cephfs/home/users/fshahi/data/FSHAHI/Mesothelioma_2005/Meso_187/40X/MESO_187_29'


total 95595880
-rwxr-xr-x 1 fshahi 4234418813 Aug 15  2018 MESO_187_29.ndpi*
-rwxr-xr-x 1 fshahi 4188044700 Aug 15  2018 MESO_187_30.ndpi*
-rwxr-xr-x 1 fshahi 5224312449 Aug 15  2018 MESO_187_31.ndpi*
-rwxr-xr-x 1 fshahi 5790635385 Aug 15  2018 MESO_187_32.ndpi*
-rwxr-xr-x 1 fshahi 5330053179 Aug 15  2018 MESO_187_33.ndpi*
-rwxr-xr-x 1 fshahi      18475 Aug 16  2018 MESO_187_33.ndpi.ndpa*
-rwxr-xr-x 1 fshahi 5862766319 Aug 15  2018 MESO_187_34.ndpi*
-rwxr-xr-x 1 fshahi       8494 Aug 16  2018 MESO_187_34.ndpi.ndpa*
-rwxr-xr-x 1 fshahi 4130432567 Aug 15  2018 MESO_187_37.ndpi*
-rwxr-xr-x 1 fshahi 6647958666 Aug 15  2018 MESO_187_38.ndpi*
-rwxr-xr-x 1 fshahi      12538 Sep 24  2018 MESO_187_38.ndpi.ndpa*
-rwxr-xr-x 1 fshahi 4627353477 Aug 15  2018 MESO_187_39.ndpi*
-rwxr-xr-x 1 fshahi 6512424848 Aug 15  2018 MESO_187_40.ndpi*
-rwxr-xr-x 1 fshahi       9035 Sep 11  2018 MESO_187_40.ndpi.ndpa*
-rwxr-xr-x 1 fshahi 6129428976 Aug 15  2018 MESO_187_41.ndpi*
-rwxr-xr-x 1 fshahi 6456691780 Aug 

In [5]:
from openslide import OpenSlide
# import pyvips
from aicsimageio import AICSImage

In [12]:
img = AICSImage('/mnt/cephfs/home/users/fshahi/Projects/Histomorphological-Phenotype-Learning/MesoGraph/dataset/AIME_2022_Mesothelioma_Cores/All_cores/13-A.jpg')
print(img.dims)
print(img.shape)
print(img.metadata)


<Dimensions [T: 1, C: 1, Z: 1, Y: 2854, X: 2854, S: 3]>
(1, 1, 1, 2854, 2854, 3)
{'jfif': 258, 'jfif_version': (1, 2), 'jfif_unit': 0, 'jfif_density': (1, 1), 'mode': 'RGB', 'shape': (2854, 2854)}


In [8]:
print(len(np.unique(cases)))
print(len(np.unique(slides)))
print(len(slides))


486
3446
3239939


In [20]:
# reading all files in a folder in specific format
import os
import glob
import pandas as pd
os.chdir("/mnt/cephfs/home/users/fshahi/Projects/Histomorphological-Phenotype-Learning/results/BarlowTwins_3/Meso/h224_w224_n3_zdim128/750K/adatas/")
extension = 'csv'
all_filenames = [i for i in glob.glob('Meso_*.{}'.format(extension))]
for file in all_filenames:
    df = pd.read_csv(file)
    df['case_Id'] = df['samples']
    df.to_csv(file, index=False)

In [7]:
import h5py
import numpy as np
dataset = 'complete'
name = 'TCGA_MESO'
# h5_path = '/mnt/cephfs/home/users/fshahi/Projects/Datasets/Meso_large/he/patches_h224_w224/hdf5_Meso_train.h5'
# h5_path = '/mnt/cephfs/home/users/fshahi/Projects/Datasets/{}/he/patches_h224_w224/hdf5_{}_he_{}.h5'.format(name,name,dataset)
# h5_file = h5py.File('/mnt/cephfs/home/users/fshahi/data/FSHAHI/Mesothelioma_2005/Meso_187/40X/MESO_187_29/MESO_187_29.h5', 'r')
# h5_file.keys()
total_patients = []
total_slides = []
# h5_path = '/mnt/cephfs/home/users/fshahi/Projects/Datasets/{}/he/patches_h224_w224/hdf5_{}_he_{}.h5'.format(name,name,dataset)
h5_path = '/mnt/cephfs/home/users/fshahi/Projects/Histomorphological-Phenotype-Learning/results/BarlowTwins_3/{}/h224_w224_n3_zdim128/hdf5_{}_he_{}_filtered_metadata.h5'.format(name,name,dataset)
with h5py.File(h5_path, 'r') as h5_file:
    # slides = h5_file['slides'][:].astype(str)
    # cases = [slide.split('_')[1] for slide in slides]

    for key in h5_file.keys():
        print(key, '_______________________________________')
        print(h5_file[key].shape)
        print(h5_file[key].dtype)
        print(h5_file[key][0])
        print(h5_file[key][-1])

# h5_file = h5py.File(h5_path, 'a')
# del h5_file['case_Id']
# h5_file.create_dataset('case_Id', data=cases, dtype='S23')

h5_file.close()






Meso_type _______________________________________
(72241,)
int64
0
0
age_at_diagnosis _______________________________________
(72241,)
int64
23002
24176
age_at_index _______________________________________
(72241,)
int64
62
66
ajcc_pathologic_m _______________________________________
(72241,)
object
b'M0'
b'M1'
ajcc_staging_system_edition _______________________________________
(72241,)
object
b'7th'
b'7th'
case_Id _______________________________________
(72241,)
|S23
b'TCGA-UT-A88C'
b'TCGA-YS-AA4M'
days_to_birth _______________________________________
(72241,)
int64
-23002
-24176
days_to_last_follow_up _______________________________________
(72241,)
object
b'723'
b"'--"
ethnicity _______________________________________
(72241,)
object
b'not hispanic or latino'
b'not reported'
gender _______________________________________
(72241,)
object
b'male'
b'male'
hist_subtype _______________________________________
(72241,)
|S37
b'train'
b'test'
img_h_latent ___________________________________

In [72]:
# Reading two hdf5 files and merging them on the common keys
import h5py
import numpy as np
h1 = h5py.File('/mnt/cephfs/home/users/fshahi/Projects/Datasets/Meso_large/he/patches_h224_w224/hdf5_Meso_large_he_test.h5', 'r')
h2 = h5py.File('/mnt/cephfs/home/users/fshahi/Projects/Datasets/Meso_500/he/patches_h224_w224/hdf5_Meso_500_he_test.h5', 'r')

h3 = h5py.File('/mnt/cephfs/home/users/fshahi/Projects/Datasets/Meso_large/he/patches_h224_w224/hdf5_Meso_combine.h5', 'w')
for key in h1.keys():
    print(key)
    print(h1[key].shape)
    print(h2[key].shape)
    shape = (h1[key].shape[0]+h2[key].shape[0],)+ h1[key].shape[1:]
    h3.create_dataset(key, shape=shape, dtype=h1[key].dtype)
    h3[key][:h1[key].shape[0]] = h1[key][:]
    h3[key][h1[key].shape[0]:] = h2[key][:]
    # print(h3[key].shape)
    # print(h3[key].dtype)
    # print(h3[key][0])
    # print(h3[key][-1])
h3.close()
h1.close()
h2.close()

h3 = h5py.File('/mnt/cephfs/home/users/fshahi/Projects/Datasets/Meso_large/he/patches_h224_w224/hdf5_Meso_combine.h5', 'r')
print(h3.keys())
for key in h3.keys():
    print(key)
    print(h3[key].shape)
    print(h3[key].dtype)
    print(h3[key][0])
    print(h3[key][-1])
h3.close()


test_hist_subtype
(41098,)
(429784,)
test_img
(41098, 224, 224, 3)
(429784, 224, 224, 3)


In [8]:

img_path = '/mnt/cephfs/home/users/fshahi/data/FSHAHI/Mesothelioma_2005/Meso_187/40X/MESO_187_51.ndpi'
# img_path = '/mnt/cephfs/home/users/fshahi/data/FSHAHI/Mesothelioma_2005/Meso_187/40X/MESO_187_29.ndpi'
# slide = OpenSlide(img_path,)
img = AICSImage(img_path, dask_tiles=True)
print(img.shape)
# image = pyvips.Image.new_from_file(img_path)
# image = large_image_source_openslide.open(img_path)


# core_im = imageio.imread(img_path)
# print(core_im.shape)

: 